In [1]:
import os

In [2]:
%pwd

'f:\\Git Projects\\Venkey-Chicken-Disease-Classification\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'f:\\Git Projects\\Venkey-Chicken-Disease-Classification'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class AWSConfig:
    access_key_id: str
    secret_access_key: str
    region_name: str

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    bucket_name: str
    file_key: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from ChickenDiseaseClassifier.constants import *
from ChickenDiseaseClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir= config.root_dir,
            bucket_name= config.bucket_name,
            file_key= config.file_key,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from ChickenDiseaseClassifier import logger
from ChickenDiseaseClassifier.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig, aws_config):
        self.config = config
        self.aws_config = aws_config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            s3_client = boto3.client(
                "s3",
                aws_access_key_id=self.aws_config.access_key_id,
                aws_secret_access_key=self.aws_config.secret_access_key,
                region_name=self.aws_config.region_name
            )

            s3_client.download_file(
                Bucket=self.config.bucket_name,
                Key=self.config.file_key,
                Filename=str(self.config.local_data_file)
            )
            logger.info(f"{self.config.local_data_file} downloaded from S3.")
        else:
            logger.info(f"File already exists of size: {get_size(self.config.local_data_file)}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logger.info(f"Extracted files to: {unzip_path}")


In [ ]:
try:
    config = ConfigurationManager()
    aws_config = config.get_aws_config()  # You’d create this method similar to get_data_ingestion_config
    data_ingestion_config = config.get_data_ingestion_config()

    data_ingestion = DataIngestion(
        config=data_ingestion_config,
        aws_config=aws_config
    )
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e
